<a href="https://colab.research.google.com/github/conglyne222/Graph_Mining/blob/main/Girvan_Newman_NetworkX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import networkx as nx
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms.community.centrality import girvan_newman
from networkx.algorithms import community
import operator

#Download data

In [ ]:
!wget https://raw.githubusercontent.com/mansiganatra/Girvan-Newman-Implementation-using-Spark/master/ub_sample_data.csv

--2022-06-15 06:05:55--  https://raw.githubusercontent.com/mansiganatra/Girvan-Newman-Implementation-using-Spark/master/ub_sample_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1777828 (1.7M) [text/plain]
Saving to: ‘ub_sample_data.csv.1’

ub_sample_data.csv. 100%[===================>]   1.70M  --.-KB/s    in 0.06s   

2022-06-15 06:05:55 (29.3 MB/s) - ‘ub_sample_data.csv.1’ saved [1777828/1777828]



# NetworkX/Girvan-Newman Functions

In [ ]:
import numpy as np
import networkx as nx

"""Functions for computing communities based on centrality notions."""

import networkx as nx

__all__ = ["girvan_newman"]


def girvan_newman(G, most_valuable_edge=None):
    """Finds communities in a graph using the Girvan–Newman method.
    Parameters
    ----------
    G : NetworkX graph
    most_valuable_edge : function
        Function that takes a graph as input and outputs an edge. The
        edge returned by this function will be recomputed and removed at
        each iteration of the algorithm.
        If not specified, the edge with the highest
        :func:`networkx.edge_betweenness_centrality` will be used.
    Returns
    -------
    iterator
        Iterator over tuples of sets of nodes in `G`. Each set of node
        is a community, each tuple is a sequence of communities at a
        particular level of the algorithm.
    Examples
    --------
    To get the first pair of communities::
        >>> G = nx.path_graph(10)
        >>> comp = girvan_newman(G)
        >>> tuple(sorted(c) for c in next(comp))
        ([0, 1, 2, 3, 4], [5, 6, 7, 8, 9])
    To get only the first *k* tuples of communities, use
    :func:`itertools.islice`::
        >>> import itertools
        >>> G = nx.path_graph(8)
        >>> k = 2
        >>> comp = girvan_newman(G)
        >>> for communities in itertools.islice(comp, k):
        ...     print(tuple(sorted(c) for c in communities))
        ...
        ([0, 1, 2, 3], [4, 5, 6, 7])
        ([0, 1], [2, 3], [4, 5, 6, 7])
    To stop getting tuples of communities once the number of communities
    is greater than *k*, use :func:`itertools.takewhile`::
        >>> import itertools
        >>> G = nx.path_graph(8)
        >>> k = 4
        >>> comp = girvan_newman(G)
        >>> limited = itertools.takewhile(lambda c: len(c) <= k, comp)
        >>> for communities in limited:
        ...     print(tuple(sorted(c) for c in communities))
        ...
        ([0, 1, 2, 3], [4, 5, 6, 7])
        ([0, 1], [2, 3], [4, 5, 6, 7])
        ([0, 1], [2, 3], [4, 5], [6, 7])
    To just choose an edge to remove based on the weight::
        >>> from operator import itemgetter
        >>> G = nx.path_graph(10)
        >>> edges = G.edges()
        >>> nx.set_edge_attributes(G, {(u, v): v for u, v in edges}, "weight")
        >>> def heaviest(G):
        ...     u, v, w = max(G.edges(data="weight"), key=itemgetter(2))
        ...     return (u, v)
        ...
        >>> comp = girvan_newman(G, most_valuable_edge=heaviest)
        >>> tuple(sorted(c) for c in next(comp))
        ([0, 1, 2, 3, 4, 5, 6, 7, 8], [9])
    To utilize edge weights when choosing an edge with, for example, the
    highest betweenness centrality::
        >>> from networkx import edge_betweenness_centrality as betweenness
        >>> def most_central_edge(G):
        ...     centrality = betweenness(G, weight="weight")
        ...     return max(centrality, key=centrality.get)
        ...
        >>> G = nx.path_graph(10)
        >>> comp = girvan_newman(G, most_valuable_edge=most_central_edge)
        >>> tuple(sorted(c) for c in next(comp))
        ([0, 1, 2, 3, 4], [5, 6, 7, 8, 9])
    To specify a different ranking algorithm for edges, use the
    `most_valuable_edge` keyword argument::
        >>> from networkx import edge_betweenness_centrality
        >>> from random import random
        >>> def most_central_edge(G):
        ...     centrality = edge_betweenness_centrality(G)
        ...     max_cent = max(centrality.values())
        ...     # Scale the centrality values so they are between 0 and 1,
        ...     # and add some random noise.
        ...     centrality = {e: c / max_cent for e, c in centrality.items()}
        ...     # Add some random noise.
        ...     centrality = {e: c + random() for e, c in centrality.items()}
        ...     return max(centrality, key=centrality.get)
        ...
        >>> G = nx.path_graph(10)
        >>> comp = girvan_newman(G, most_valuable_edge=most_central_edge)
    Notes
    -----
    The Girvan–Newman algorithm detects communities by progressively
    removing edges from the original graph. The algorithm removes the
    "most valuable" edge, traditionally the edge with the highest
    betweenness centrality, at each step. As the graph breaks down into
    pieces, the tightly knit community structure is exposed and the
    result can be depicted as a dendrogram.
    """
    # If the graph is already empty, simply return its connected
    # components.
    if G.number_of_edges() == 0:
        yield tuple(nx.connected_components(G))
        return
    # If no function is provided for computing the most valuable edge,
    # use the edge betweenness centrality.
    if most_valuable_edge is None:

        def most_valuable_edge(G):
            """Returns the edge with the highest betweenness centrality
            in the graph `G`.
            """
            # We have guaranteed that the graph is non-empty, so this
            # dictionary will never be empty.
            betweenness = nx.edge_betweenness_centrality(G)
            return max(betweenness, key=betweenness.get)

    # The copy of G here must include the edge weight data.
    g = G.copy().to_undirected()
    # Self-loops must be removed because their removal has no effect on
    # the connected components of the graph.
    g.remove_edges_from(nx.selfloop_edges(g))
    while g.number_of_edges() > 0:
        yield _without_most_central_edges(g, most_valuable_edge)



def _without_most_central_edges(G, most_valuable_edge):
    """Returns the connected components of the graph that results from
    repeatedly removing the most "valuable" edge in the graph.
    `G` must be a non-empty graph. This function modifies the graph `G`
    in-place; that is, it removes edges on the graph `G`.
    `most_valuable_edge` is a function that takes the graph `G` as input
    (or a subgraph with one or more edges of `G` removed) and returns an
    edge. That edge will be removed and this process will be repeated
    until the number of connected components in the graph increases.
    """
    original_num_components = nx.number_connected_components(G)
    num_new_components = original_num_components
    while num_new_components <= original_num_components:
        edge = most_valuable_edge(G)
        G.remove_edge(*edge)
        new_components = tuple(nx.connected_components(G))
        num_new_components = len(new_components)
    return new_components

#Read Data

In [ ]:
data = []
header = []
with open('ub_sample_data.csv', newline ='') as csvfile:
    file = csv.reader(csvfile, delimiter=',')
    for row in file:
        if not header:
            header.append(row)
            continue
        data.append(row)
    csvfile.close()

# 4.1 Betweeness Calculation

## Set threshold & make graph from the dataset

In [ ]:
threshold = 7
users = {}

# add user_id and business_id to dict users
for [u, i] in data:
    if u not in users:
        users[u] = {i}
    else:
        users[u].add(i)

g = {}

# make graph based on threshold (>=7)
# filter and make adjacency list of graph
for u1, i1 in users.items():
    for u2, i2 in users.items():
        if u1 == u2: continue
        if(threshold <= len(i1 & i2)):
            if u1 not in g:
                g[u1] = [u2]
            else:
                g[u1].append(u2)

In [ ]:
G = nx.Graph(g)
bc = nx.edge_betweenness_centrality(G)

## Check number of vertices and edges

In [ ]:
print(G)

Graph with 222 nodes and 498 edges


---------------------------------------------
Sort the result based on requirements:

*   Sort by betweeness_centrality descending
*   Sort by key user_id1, then user_id2 in lexicographical order



In [ ]:
sorted_bc = sorted(bc.items(), key=lambda x: (x[1],x[0]), reverse = True)

In [ ]:
sorted_bc

[(('cyuDrrG5eEK-TZI867MUPA', 'l-1cva9rA8_ugLrtSdKAqA'), 0.17259793730381964),
 (('DKolrsBSwMTpTJL22dqJRQ', '1st2ltGKJ00ZcRsev-Ieew'), 0.16250814486108625),
 (('1st2ltGKJ00ZcRsev-Ieew', 'HLY9oDcVBH9D25lU4X_V5Q'), 0.15722962781786312),
 (('1st2ltGKJ00ZcRsev-Ieew', 'Hv_q_ZnSIoZwdcoH0CyV2Q'), 0.15180791651379885),
 (('JM0GL6Dx4EuZ1mprLk5Gyg', '1st2ltGKJ00ZcRsev-Ieew'), 0.14958675546910818),
 (('HLY9oDcVBH9D25lU4X_V5Q', 'l-1cva9rA8_ugLrtSdKAqA'), 0.1359504300680771),
 (('l-1cva9rA8_ugLrtSdKAqA', 'Hv_q_ZnSIoZwdcoH0CyV2Q'), 0.13407525172231055),
 (('0FVcoJko1kfZCrJRfssfIA', 'DKolrsBSwMTpTJL22dqJRQ'), 0.09035984963113668),
 (('a48HhwcmjFLApZhiax41IA', 'o-t-i7nbT5N_cmkCXs5oDQ'), 0.08026578614813909),
 (('A-U-K9z9oraMH7eBZW1dOA', 'l-1cva9rA8_ugLrtSdKAqA'), 0.07737148913619502),
 (('cyuDrrG5eEK-TZI867MUPA', 'o-t-i7nbT5N_cmkCXs5oDQ'), 0.0759039582568994),
 (('39FT2Ui8KUXwmUt6hnwy-g', 'JM0GL6Dx4EuZ1mprLk5Gyg'), 0.057106576137476904),
 (('JM0GL6Dx4EuZ1mprLk5Gyg', 'KLB3wIYUwKDPMbijIE92vg'), 0.0533249

Save output file of betweenees calculation

In [ ]:
with open('betweenness.txt', "w+") as fp:
    for i in sorted_bc:
        string_to_write = str(i[0])
        string_to_write += ", " + str(i[1])
        fp.write(string_to_write)
        fp.write("\n")

    fp.close()

# 4.2 Community Detection with Girvan Newman algorithm

Apply girvan_newman function of NetworkX lib for graph G

In [ ]:
com = community.girvan_newman(G)
com_lst = list(com)

## Detect community with global highest modularity

Set paRam - Resolution = 1 (Popular choice)

In [ ]:
highest_modularity = 0
communities = []
modularity_nx = []
for i in com_lst:
  modularity = community.modularity(G, i, resolution = 1)
  modularity_nx.append(modularity)
  #print(modularity)

  if modularity > highest_modularity:
    highest_modularity = modularity
    communities = [sorted(x) for x in i]

Max modularity

In [ ]:
print(highest_modularity)

0.6872550442734795


Number of community

In [ ]:
len(communities)

19

-------------------------------------------------------------
Sort communities based on requirements:
  * Sort the communities ascendingly by their sizes
  * Sort by user_id1, then user_id2 in lexicographical order

In [ ]:
#Sort communities by lenght of nodes & lexicographical
communities.sort(key = lambda x: (len(x),x))

In [ ]:
community_size = {}
for i in communities:
  length = len(i)
  if length not in community_size:
    community_size[length] = 1
  else:
    community_size[length] += 1

## Display size of communities (1)

In [ ]:
community_size

{2: 8, 3: 2, 4: 2, 5: 1, 6: 1, 11: 1, 14: 1, 33: 1, 46: 1, 77: 1}

Save output file of new community

In [ ]:
file = open("gm_communities1.txt", "w")
for i in communities:
  for j in i:
    file.write("'" + str(j) + "',")
  file.seek(file.tell()-1)
  file.write("\n")
file.close()

Set paRam - Resolution = 3 (For dividing community with 1 node)

In [ ]:
highest_modularity = 0
communities1 = []
modularity_nx = []
for i in com_lst:
  modularity = community.modularity(G, i, resolution = 3)
  modularity_nx.append(modularity)
  #print(modularity)

  if modularity > highest_modularity:
    highest_modularity = modularity
    communities1 = [sorted(x) for x in i]

Max modularity

In [ ]:
print(highest_modularity)

0.31512193351720136


Number of community

In [ ]:
len(communities1)

36

<center>Highest Modularity for 5 resolutions


Resolution | Highest Modularity
--- | ---
1   | 0.6872550442734795
2   | 0.47380284188964694
3   | 0.31512193351720136
4   | 0.1630054353962033
5   | 0.04858187771164977